In [2]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载音频处理库

In [3]:
# 其他库

import os
import librosa
import librosa.display
import glob 

## 特征提取以及数据集的建立

In [4]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}
print(label_dict)
print(label_dict_inv)

{'aloe': 0, 'burger': 1, 'cabbage': 2, 'candied_fruits': 3, 'carrots': 4, 'chips': 5, 'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream': 11, 'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon': 17, 'soup': 18, 'wings': 19}
{0: 'aloe', 1: 'burger', 2: 'cabbage', 3: 'candied_fruits', 4: 'carrots', 5: 'chips', 6: 'chocolate', 7: 'drinks', 8: 'fries', 9: 'grapes', 10: 'gummies', 11: 'ice-cream', 12: 'jelly', 13: 'noodles', 14: 'pickles', 15: 'pizza', 16: 'ribs', 17: 'salmon', 18: 'soup', 19: 'wings'}


In [5]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
            # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            # print(fn)
            label_name = fn.split('/')[-2]
            # print(label_name)
            
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
    
    return [feature, label]



In [6]:
# 自己更改目录  
parent_dir = '../data/train_sample/'
save_dir = "../data"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label

temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:05<00:00,  6.53it/s]


In [7]:
temp = np.array(temp)
data = temp.transpose()
# 转置
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1],dtype= int)
print('X的特征尺寸是',X.shape)
print('Y的特征尺寸是',Y.shape)
from tensorflow.keras.utils import to_categorical 

# Y = to_categorical(Y)

/tmp/ipykernel_150583/1712219130.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp = np.array(temp)


X的特征尺寸是 (1000, 128)
Y的特征尺寸是 (1000,)


2022-09-06 13:42:47.671812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/anaconda3/lib:/cm/shared/apps/hdf5/1.10.1/lib:/cm/local/apps/cuda/libs/current/lib64:/cm/shared/apps/cuda10.2/toolkit/10.2.89/targets/x86_64-linux/lib:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64:/cm/shared/apps/gcc8/8.4.0/lib:/cm/shared/apps/gcc8/8.4.0/lib32:/cm/shared/apps/gcc8/8.4.0/lib64
2022-09-06 13:42:47.671863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [30]:
# 最终数据
print(X.shape)
# X = X.reshape(-1, 1, 16, 8, 1)
X = X.reshape(-1,1, 1, 16, 8)
Y = Y.reshape(-1,1)
print(X.shape)
print(Y.shape)
print(Y.dtype)



(1000, 1, 1, 16, 8)
(1000, 1, 1, 16, 8)
(1000, 1)
int64


In [31]:
import torch
import torchvision
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# from torchvision import transforms
import matplotlib.pyplot as plt
import sys, os
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [32]:
b = torch.from_numpy(X)
#c = Y
c = torch.from_numpy(Y)
print(b.size())
X_train, X_test, y_train, y_test = train_test_split(b, c, test_size=0.1, random_state=42)
print(y_train[0])


torch.Size([1000, 1, 1, 16, 8])
tensor([13])


In [33]:
# 基于分布式光纤传感器的

In [ ]:
class CNN2D(nn.module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(1,6,3)
        self.bn1 = nn.BatchNorm1d()
        self.maxpool1 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(6,25,5)
        self.conv4 = nn.Conv2d(6,25,5)
        


In [67]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,64,3,padding=2)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64,128,3,padding=2)
        # self.conv3 = nn.Conv2d(128,16,3,padding=2)
        self.avgpool = nn.AvgPool2d(kernel_size=2,stride=2)
        self.fla = nn.Flatten()
        self.fc1 = nn.Linear(240*8,1024)
        self.fc2 = nn.Linear(1024,20)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = self.pool(x)
        # tanh 较好
        x = self.conv2(x)
        x = F.tanh(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.3,training=self.training)
        # x = x.view(-1, (28*28)//(4*4)*8)
        x = x.view(-1,240*8)
        x = self.fc1(x)
        x = F.tanh(x)

        x = F.dropout(x, p=0.2,training=self.training)
        x = self.fc2(x)
        # x = sfmax(x)
        return x


In [68]:
predicted = torch.tensor([[1,2,3,4]]).float()
target = torch.tensor([1]).long()
lossfxn = nn.CrossEntropyLoss() # 已经集成一部分softmax，无需重复使用
loss = lossfxn(predicted, target)
print(loss) # outputs tensor(2.4402)
print(predicted)
print(target)

tensor(2.4402)
tensor([[1., 2., 3., 4.]])
tensor([1])


In [69]:
net = model()
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(net.parameters(),lr = 0.001,momentum=0.9)
print(b.shape)
for epoch in range(20):
    for i,da in enumerate(X_train):
        x = da
        #print(x)
        y = y_train[i]

        # print(y)
        net.train() #打开dropout
        pred = net(x)
        #print(pred)
        loss = loss_fn(pred,y)
        # print(i,loss)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(epoch,loss)

torch.Size([1000, 1, 1, 16, 8])
0 tensor(2.5723, grad_fn=<NllLossBackward>)
1 tensor(2.4033, grad_fn=<NllLossBackward>)
2 tensor(1.8380, grad_fn=<NllLossBackward>)
3 tensor(1.8075, grad_fn=<NllLossBackward>)
4 tensor(1.1560, grad_fn=<NllLossBackward>)
5 tensor(1.0311, grad_fn=<NllLossBackward>)
6 tensor(1.0193, grad_fn=<NllLossBackward>)
7 tensor(0.5636, grad_fn=<NllLossBackward>)
8 tensor(0.8501, grad_fn=<NllLossBackward>)
9 tensor(0.1371, grad_fn=<NllLossBackward>)
10 tensor(0.1583, grad_fn=<NllLossBackward>)
11 tensor(0.3896, grad_fn=<NllLossBackward>)
12 tensor(0.0490, grad_fn=<NllLossBackward>)
13 tensor(0.1559, grad_fn=<NllLossBackward>)
14 tensor(0.3382, grad_fn=<NllLossBackward>)
15 tensor(0.0084, grad_fn=<NllLossBackward>)
16 tensor(0.0090, grad_fn=<NllLossBackward>)
17 tensor(0.0682, grad_fn=<NllLossBackward>)
18 tensor(0.0153, grad_fn=<NllLossBackward>)
19 tensor(0.0082, grad_fn=<NllLossBackward>)


In [70]:
def accuracy(a,b):
    acc  = 0
    for i,da in enumerate(a):
        x = da
        # print(x)
        y = b[i]
        pred = net(x)
        pred = pred
        # print(pred)
        m = max(pred[0])
        # print(m)
        pre_list = pred[0].tolist()
        ind = pre_list.index(m)
        
        y_list = y.tolist()
        #print(y_list)
        # y_max = max(y_list)
        # y_ind = y_list.index(y_max)

        if ind == y_list[0]:
            acc += 1
        # print(ind, y_ind)
    print(acc/len(a))

In [71]:
accuracy(X_test,y_test)
accuracy(X_train,y_train)


0.42
0.7466666666666667
